In [8]:
# make csv ready to datasets fromat
import pandas as pd
from transformers import TrainingArguments, Trainer
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
import numpy as np


In [ ]:
#prepare data for model
d_a = pd.read_csv("data/twitter_data/new_tweet_dataset_train.csv")
d_a = d_a.rename(columns={'tweet_text':'Tweet'})
d_a['billed'] = d_a['Tweet'].apply(lambda x: 1 if 'http' in x else 0)
d_a_ub = d_a[d_a['billed']==0]
d_a_ub[["Tweet","label"]].to_csv("data/twitter_data/new_tweet_train.csv",index=False)

In [71]:
data_files = {"train": "data/twitter_data/new_tweet_train.csv"}
dataset = load_dataset("csv", data_files=data_files)

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["Tweet"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

model = AutoModelForSequenceClassification.from_pretrained("trained_bert_3",local_files_only=True, num_labels=2)
trainer = Trainer(model)

# %% Prediction 
testset = tokenized_datasets["train"].select(range(200))
predictions, labels, _, = trainer.predict(testset)



Using custom data configuration default-16709c5269535670
Found cached dataset csv (C:/Users/bruger/.cache/huggingface/datasets/csv/default-16709c5269535670/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-16709c5269535670
Found cached dataset csv (C:/Users/bruger/.cache/huggingface/datasets/csv/default-16709c5269535670/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

loading configuration file config.json from cache at C:\Users\bruger/.cache\huggingface\hub\models--bert-base-cased\snapshots\5532cc56f74641d4bb33641f5c76a55d11f846e0\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file vocab.txt from cache at C:\Users\bruger/.cache\huggingface\hub\models--bert-base-cased\snapshots\5532cc56f74641d4bb33641f5c76a55d11f846e0\vocab.txt
l

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

loading configuration file trained_bert_3\config.json
Model config BertConfig {
  "_name_or_path": "trained_bert_3",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file trained_bert_3\pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were 

  0%|          | 0/25 [00:00<?, ?it/s]

In [76]:
#accuarcy
np.sum(np.argmax(predictions,axis=-1)==labels)/200

0.825

In [107]:
#examples
testset['Tweet'][:2], testset['Tweet'][175],testset['Tweet'][45]

(['lmaooo!!!!!!“@Bankroll_Bump #POWPOW U DUM FAT HOE “@ms_dominicana FUCK LIL PHAT IF YOU ASK ME DUMB ASS NIGGER NAME IDGAF WHO MAD””',
  '@TheDebbyRyan Hello I love so much ! Please follow :)'],
 '@MisGrace @GBabeuf @RJennromao @DavidJo52951945 @Novorossiyan @gbazov @NewsCoverUp @rougek68 Oh, good, the Putin trolls are all showing up.',
 'Turner did not withhold his disappointment. Turner called the court an “abominable conclave of negro hating demons” (with one exception) who “issued another decree that colored men and women must be driven into Jim Crow cars whenever it suits the whim of any white community.”')